In [1]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import col, count, avg, isnan, when, lit, monotonically_increasing_id, split

In [ ]:
import sys
print(sys.executable)

------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Iniciamos Sesión con Ophelia

In [2]:
import visualize
from ophelia.ophelib.OpheliaMain import Ophelia

In [3]:
ophelia = Ophelia("AssetPriceMaster")

00:28:56.200 Ophelia [TAPE] +---------------------------------------------------------------------+
00:28:56.201 Ophelia [INFO] | My name is Ophelia Vendata                                          |
00:28:56.201 Ophelia [INFO] | I am an artificial assistant for data mining & ML engine with spark |
00:28:56.201 Ophelia [INFO] | Welcome to Ophelia spark miner engine                               |
00:28:56.201 Ophelia [INFO] | Lib Version Ophelia.dev1.0                                          |
00:28:56.201 Ophelia [WARN] | V for Vendata...                                                    |
00:28:56.201 Ophelia [TAPE] +---------------------------------------------------------------------+
00:28:56.201 Ophelia [WARN] Initializing Spark Session
00:29:05.307 Ophelia [INFO] Spark Version: 3.0.0
00:29:05.307 Ophelia [INFO] This Is: 'AssetPriceMaster' App


# Loading CSV daily price Funds file.

In [4]:
spark = ophelia.SparkSession

In [ ]:
split_string = split(col('TICKER'), ' ')
spark.readFile('data/staging/benchmark/ticker_name', 'parquet').select("*", 
                                                                       split_string.getItem(0).alias('ticker'),
                                                                       split_string.getItem(1).alias('ticker_market'),
                                                                       split_string.getItem(2).alias('ticker_asset'),
                                                                       (monotonically_increasing_id() + 10).alias('key_id')).show(5, False)

In [ ]:
def read_data(path_file, source, date_col, withSchema=True):
    load_data = spark.readFile(path_file, source)
    if withSchema is True:
        return ophelia.Read.build_portfolio_schema(portfolio_data, date_col)
    return portfolio_data

In [55]:
portfolio_df = spark.readFile('data/staging/benchmark/close_historic_price', 'parquet')
portfolio_df.describe().show(5, False)
portfolio_df.show(5, False)

23:50:58.714 Ophelia [INFO] Read Parquet File From Path: ['data/staging/benchmark/close_historic_price']


Py4JJavaError: An error occurred while calling o664.parquet.
: java.lang.ClassCastException: class java.util.ArrayList cannot be cast to class java.lang.String (java.util.ArrayList and java.lang.String are in module java.base of loader 'bootstrap')
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:44)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:361)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:279)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:268)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:268)
	at org.apache.spark.sql.DataFrameReader.parquet(DataFrameReader.scala:737)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/

## Change impure schema portfolio input data.

In [ ]:
portfolio_df = read_data(
    path_file="data/raw/csv/data.csv",
    source="csv",
    date_col="operation_date",
)

portfolio_df.limit(5).toPandas()

In [ ]:
from com.ophelia.wrapper import SparkWrapper
from com.ophelia.utils import ListUtils

In [ ]:
@staticmethod
def build_portfolio_schema(df: DataFrame, date_col: str) -> DataFrame:
    """
    Build portfolio schema helps to generate schema data type for fund portfolio price
    :param df: spark DataFrame with fund price daily
    :param date_col: str name for DataFrame date column
    """
    if df is None or date_col is None:
        Read.__logger.error("build portfolio schema need 'df' and 'date col' not be None")
        raise TypeError("params must not be None")
    Read.__logger.info("Build Schema File")
    build_date = date_format(to_date(col(df.columns[0]), 'dd/MM/yyyy'), "yyyy-MM-dd")
    schema_portfolio = [build_date.cast("date").alias(date_col)] + [col(x).cast('float') for x in df.columns[1:]]
    schema_df = df.where(col(df.columns[0]).isNotNull()).select(schema_portfolio)
    Read.__logger.info("Build Schema Success")
    return schema_df

In [6]:
class NullDebug(object):
    
    @staticmethod
    def cleansing_list(self, partition_by: str, offset: float = 0.5):
        if partition_by is None:
            raise TypeError(f"'partition_by' required parameter, invalid {partition_by} input.")
        return self.toNarrow(partition_by, ['pivot', 'value']).groupBy('pivot')\
                   .agg(count(when(isnan('value') | col('value').isNull(), 'value')).alias('null_count'))\
                   .select('*', (col('null_count') / self.Shape[0]).alias('null_pct'))\
                   .where(col('null_pct') <= offset).uniqueRow('pivot')

    @staticmethod
    def null_clean(self, partition_by=None, offset=0.5):
        if partition_by is None:
            gen_part = self.select(monotonically_increasing_id().alias('partition_id'), "*")
            cleansing_list = NullDebug.cleansing_list(gen_part, 'partition_id', offset)
            return gen_part.select('partition_id', *cleansing_list)
        cleansing_list = NullDebug.cleansing_list(gen_partition, partition_by, offset)
        return self.select(partition_by, *cleansing_list)

DataFrame.nullClean = NullDebug.null_clean

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit, explode, struct, array, round as spark_round

def panel_format(self, fix_col, new_cols=None):
    if isinstance(new_cols, str):
        pivot_col, value_col = new_cols.split(',')[0], new_cols.split(',')[1]
    elif new_cols is not None and isinstance(new_cols, list):
        pivot_col, value_col = new_cols[0], new_cols[1]
    else:
        pivot_col, value_col = 'no_name_pivot_col', 'no_name_value_col'
    cols, dtype = zip(*[(c, t) for (c, t) in self.dtypes if c not in [fix_col]])
    
    if len(set(map(lambda x: x[-1], dtype[1:]))) != 1:
        raise AssertionError("Columns must be of the same datatype")
    
    generator_explode = explode(array([
        struct(lit(c).alias(pivot_col), col(c).alias(value_col)) for c in cols
    ])).alias('column_explode')
    column_to_explode = [f'column_explode.{pivot_col}', f'column_explode.{value_col}']
    
    return self.select([fix_col] + [generator_explode]).select([fix_col] + column_to_explode)

In [ ]:
#panel_format(portfolio_df.drop('close_date', 'close_year'), 'close_timestamp').orderBy(col('close_timestamp')).show()

In [ ]:
#portfolio_df.drop('close_date', 'close_year').toPanel('close_timestamp').show()

In [ ]:
#NullDebug.cleansing_list(portfolio_df.drop('close_date', 'close_year'), 'close_timestamp')

In [7]:
portfolio_df.drop('close_timestamp', 'close_date', 'close_year').Shape

(4643, 71)

In [8]:
portfolio_df.drop('close_timestamp', 'close_date', 'close_year').nullClean(offset=0.1).Shape

(4643, 66)

In [ ]:
from pyspark.sql import DataFrame, Window
from pyspark.sql.functions import (when, row_number, lit, count, first, sum as spark_sum,
                                   min as spark_min, max as spark_max, mean, stddev, variance)

def spark_methods():
    return {
        'sum': spark_sum,
        'min': spark_min,
        'max': spark_max,
        'mean': mean,
        'stddev': stddev,
        'var': variance,
        'first': first,
        'count': count,
    }

def rolling_down(self, op_col, nat_order, min_p=2, window=2, method='sum'):
    w = Window.orderBy(nat_order).rowsBetween(
        Window.currentRow - (window - 1), Window.currentRow)
    if isinstance(op_col, list):
        rolling = [spark_methods()[method](c).over(w).alias(f'{c}_rolling_{method}') for c in op_col]
        return self.select(*rolling)
    if method == 'count':
        if isinstance(op_col, list):
            rolling = [spark_methods()[method](c).over(w).alias(f'{c}_rolling_{method}') for c in op_col]
            return self.select(*rolling)
        rolling = spark_methods()[method](op_col).over(w).alias(f'{op_col}_rolling_{method}')
        return self.select('*', rolling)
    _unbounded_w = Window.orderBy(nat_order).rowsBetween(
        Window.unboundedPreceding, Window.currentRow)
    rolling = when(
        row_number().over(_unbounded_w) >= min_p,
        spark_methods()[method](op_col).over(w),
        ).otherwise(lit(None)).alias(f'{op_col}_rolling_{method}')
    return self.select('*', rolling)

In [ ]:
sel = portfolio_df.select(portfolio_df.columns[:4])
rolling_down(sel, op_col='AXESCP', nat_order='operation_date', method='mean').show()

In [ ]:
sel = portfolio_df.select(portfolio_df.columns[:4])
rolling_down(sel, op_col=['SCOTIAG'], nat_order='operation_date', method='mean').show()

In [ ]:
#portfolio_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in portfolio_df.columns[1:]]).show()

In [54]:
[date for date in pd.date_range(start="2020-02-02",end="2020-09-02")][0].

Timestamp('2020-02-02 00:00:00', freq='D')

In [41]:
import pandas as pd
from datetime import datetime
datelist = pd.date_range(datetime.today(), periods=5, freq='Y').tolist()
datelist[0].__class__

pandas._libs.tslibs.timestamps.Timestamp

## Defining Year parameters input array:

# Cleaning data, analytic base table structuration.

In [17]:
def portfolio_date_window(df, from_year, to_year, col_date):
    year_array = ophelia.ophelia_array.year_array(from_year, to_year)
    split_dates = ophelia.ophelia_df.split_date(df, col_date)
    operation_dates_list = ophelia.ophelia_array.sorted_date_list(df, col_date)
    date_index_udf = ophelia.ophelia_array.dates_index(operation_dates_list)
    portfolio_dates = split_dates.where(col(col_date+"_year").isin(year_array))\
                                 .select('*', (date_index_udf(col(col_date))).alias(col_date[:9]+"_id"))
    return portfolio_dates

In [18]:
portfolio_window_df = portfolio_date_window(
    df=portfolio_df, 
    from_year="2016", 
    to_year="2019", 
    col_date="operation_date"
)

AttributeError: 'Ophelia' object has no attribute 'ophelia_array'

In [ ]:
def monitoring_empty_vector(df, feature_type):
    float_cols = ophelia.ophelia_array.feature_picking(df)[str(feature_type)]
    count_by_col = [spark_count(col(x)).alias(str(x)) for x in float_cols]
    aggregate_columns = df.select(*count_by_col)
    return aggregate_columns

In [ ]:
def debug_null(panel, missing_days, N):
    null_count = panel.select([col(c).alias(c) for c in panel.columns]).collect()[0].asDict()
    clean_null_list = [k for k, v in null_count.items() if v < abs(missing_days - N)]
    return clean_null_list

In [ ]:
def debug_empty_vector(df, feature_type, missing_days=10):
    sample_count = df.count()
    empty_panel = monitoring_empty_vector(df, feature_type)
    clean_null_list = debug_null(empty_panel, missing_days, sample_count)
    debug_vector = df.drop(*clean_null_list)
    return debug_vector

In [ ]:
remove_none_df = debug_empty_vector(portfolio_window_df, feature_type="float")
remove_none_df.limit(5).toPandas()

In [ ]:
def mean_impute(df):
    float_cols = ophelia.ophelia_array.feature_picking(df)["float"]
    numerical_fields = df.agg(*(spark_avg(c).alias(c) for c in df.columns if c in float_cols))
    portfolio_base_table = df.na.fill(numerical_fields.collect()[0].asDict())
    return portfolio_base_table

# Now we can write our masterized analytical base table:

In [ ]:
portfolio_base_table = mean_impute(remove_none_df).drop("operation_date_year", "operation_date_month", "operation_date_day")
portfolio_base_table.orderBy(col("operation_date").desc()).limit(5).toPandas()

In [ ]:
portfolio_path = ophelia.ophelia_write.write_parquet(
    df=portfolio_base_table,
    output_type="engine",
    project="OpheliaPortfolio",
    part="operation_date"
)